# CMPT 423/820 Assignment 2 Question 3
### Model Solution and Grading Scheme

## Prologue: Importing and reading the Data

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier

from sklearn import model_selection
from sklearn.preprocessing import StandardScaler

# give names to the columns
cnames = ['class']+["f_"+str(n) for n in range(1,14)]
dataframe = pd.read_csv('a2q3.csv', 
                        header=None, 
                        names=cnames,
                        index_col=False)

# Separate the features from the class
array = dataframe.values
X = array[:,1:14]
Y = array[:,0]

# Proportion of data as a part of validation set
validation_size = 0.20

# set up 10-fold cross validation
kfold = 10

# set up the performance metric once and for all
metric_name = 'f1_macro'

## Part 1: K-Nearest Neighbours
K-Nearest Neighbours is a non-parametric classifier.  In other words, it does not try to model the task environment by simplifying it.  Instead, it remembers data, and uses a distance metric to determine if a new example is close to something it has already seen.  

#### Scaling the data
Because all the data is numeric, a Euclidean distance metric is plausible (though not necessarily the only option).  However, because distance depends on scale, we must scale the data to a standard range.  

Scaling can be done using Python and Numpy, or scikit-learn modules.  The code below uses a `StandardScaler` which scales each feature independently, so that the scaled feature has zero mean, and unit variance.  

In [2]:
X_scaled = StandardScaler().fit_transform(X)

# cross validation to evaluate the performance
cv_results = model_selection.cross_val_score(KNeighborsClassifier(n_neighbors=7), 
                                             X_scaled, Y, cv=kfold, scoring=metric_name)

#displaying the mean and standard deviation of the prediction
msg = "%s: %f (%f)" % ('KNN performance with scaling', cv_results.mean(), cv_results.std())
print(msg)


KNN performance with scaling: 0.966731 (0.036989)


#### KNN Comparison to unscaled data
To see the importance of scaling for KNN, observe the performance without scaling:

In [3]:
# cross validation to evaluate the performance 
cv_results = model_selection.cross_val_score(KNeighborsClassifier(n_neighbors=7), 
                                             X, Y, cv=kfold, scoring=metric_name)

#displaying the mean and standard deviation of the prediction
msg = "%s: %f (%f)" % ('KNN performance without scaling', cv_results.mean(), cv_results.std())
print(msg)


KNN performance without scaling: 0.645687 (0.087908)


Clearly scaling improves the KNN classifier.

In [4]:
from sklearn.model_selection import train_test_split

# use the training set to tune the vlaue of min_samples_split
X_train, X_test, Y_train, Y_test = train_test_split(X_scaled, Y, test_size=0.50)

for k in range(1,15,2):
    # cross validation to evaluate the performance 
    cv_results = model_selection.cross_val_score(KNeighborsClassifier(n_neighbors=k), 
                                                 X_train, Y_train, cv=kfold, scoring=metric_name)

    # displaying the mean and standard deviation of the prediction
    msg = "%s %d: %f (%f)" % ('KNN performance tuning k', k, cv_results.mean(), cv_results.std())
    print(msg)



KNN performance tuning k 1: 0.915873 (0.103934)
KNN performance tuning k 3: 0.940159 (0.078701)
KNN performance tuning k 5: 0.959153 (0.050271)
KNN performance tuning k 7: 0.959153 (0.050271)
KNN performance tuning k 9: 0.960000 (0.049267)
KNN performance tuning k 11: 0.958730 (0.071057)
KNN performance tuning k 13: 0.958730 (0.071057)


Now let's apply this choice of $k$ to the validation set, and get an estimate of the performance on unseen data.

In [5]:
choice_k = 7

# cross validation to evaluate the performance 
cv_results = model_selection.cross_val_score(KNeighborsClassifier(n_neighbors=choice_k), 
                                             X_test, Y_test, cv=kfold, scoring=metric_name)

#displaying the mean and standard deviation of the prediction
msg = "%s: %f (%f)" % ('KNN performance on validation set after tuning k', cv_results.mean(), cv_results.std())
print(msg)


KNN performance on validation set after tuning k: 0.988571 (0.034286)


Now let's apply this choice of $k$ to the whole data set, and get an estimate of the performance.  We're doing this just to see the difference.

In [6]:
# cross validation to evaluate the performance 
cv_results = model_selection.cross_val_score(KNeighborsClassifier(n_neighbors=choice_k), 
                                             X_scaled, Y, cv=kfold, scoring=metric_name)

#displaying the mean and standard deviation of the prediction
msg = "%s: %f (%f)" % ('KNN performance on whole data set after tuning', cv_results.mean(), cv_results.std())
print(msg)


KNN performance on whole data set after tuning: 0.966731 (0.036989)


This is pretty close to the performance score on the validation set.  It might be over-fitting a little, because we trained on half of that data.

### Grading
For full marks:
* You fitted the KNN with a deliberately chosen $k$.
   * The best way to fit $k$ is using a formal procedure like the above.
   * An informal attempt to find $k$ by giving a few values a try is less good.
* You calculated the $f_1$ score on the data
   * A validation set is the best way
   * Using cross-validation with the whole data set is less good.

## Part 2: Naive Bayes
Because Naive Bayes does not use a distance metric, we don't need to use the scaled data.  Scaling should not affect the performance at all.  Because Naive Bayes is so simple, there is no need to play around with settings and options in Scikit Learn.  Below I use the validation sets just to keep the comparison fair.

In [7]:
# cross validation to evaluate the performance 
cv_results = model_selection.cross_val_score(GaussianNB(), 
                                             X_test, Y_test, cv=kfold, scoring=metric_name)

#displaying the mean and standard deviation of the prediction
msg = "%s: %f (%f)" % ('NB performance on validation set', cv_results.mean(), cv_results.std())
print(msg)


NB performance on validation set: 0.988571 (0.034286)


Now let's try the full data set.

In [8]:
# cross validation to evaluate the performance 
cv_results = model_selection.cross_val_score(GaussianNB(), 
                                             X, Y, cv=kfold, scoring=metric_name)

#displaying the mean and standard deviation of the prediction
msg = "%s: %f (%f)" % ('NB performance on full data set', cv_results.mean(), cv_results.std())
print(msg)


NB performance on full data set: 0.978306 (0.026591)


According to our measure, the so-called $f_1$ score (combining precision and recall), Naive Bayes is doing better than KNN.

### Grading
For full marks:
* You fitted the Naive Bayes.
* You calculated the $f_1$ score on the data
   * A validation set is the best way
   * Using cross-validation with the whole data set is less good.

## Part 3: Decision Trees
The decision tree classifier in Scikit Learn has a lot of options.  You can choose:
* the purity measure, either `gini` or `entropy`
* the maximum depth of the tree
* the number of samples needed to allow a split
* lots of others!

Most of these options are used to prevent a Decision Tree from growing too big.  Big trees tend to over-fit the training data, leading to poor generalization.  We will see evidence of that here.

In the next cell, we use default options, to get a base line.

In [9]:
# cross validation to evaluate the performance 
cv_results = model_selection.cross_val_score(DecisionTreeClassifier(), 
                                             X, Y, cv=kfold, scoring=metric_name)

#displaying the mean and standard deviation of the prediction
msg = "%s: %f (%f)" % ('DT performance for default tree fitting', cv_results.mean(), cv_results.std())
print(msg)


DT performance for default tree fitting: 0.849319 (0.135809)


That's worse than the other two.  

I spent a good amount of time trying to find a good option to limit depth to avoid over-fitting.  I tried most of the Scikit Learn options at least once, and I settled on trying to prevent splits when the number of samples was too low: `min_samples_split`.  

In the cell below, I split into training and validation sets.  I try various values for `min_samples_split` and display the results.  

In [10]:
from sklearn.model_selection import train_test_split

# use the training set to tune the vlaue of min_samples_split
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.50)

for dep in range(5,100,5):
    # cross validation to evaluate the performance 
    cv_results = model_selection.cross_val_score(DecisionTreeClassifier(min_samples_split=dep), 
                                                 X_train, Y_train, cv=kfold, scoring=metric_name)

    # displaying the mean and standard deviation of the prediction
    msg = "%s %d: %f (%f)" % ('DT performance tuning min_samples_split', dep, cv_results.mean(), cv_results.std())
    print(msg)


DT performance tuning min_samples_split 5: 0.862857 (0.081602)
DT performance tuning min_samples_split 10: 0.851429 (0.068555)
DT performance tuning min_samples_split 15: 0.838651 (0.088935)
DT performance tuning min_samples_split 20: 0.851429 (0.068555)
DT performance tuning min_samples_split 25: 0.808810 (0.142027)
DT performance tuning min_samples_split 30: 0.778016 (0.132841)
DT performance tuning min_samples_split 35: 0.733519 (0.112683)
DT performance tuning min_samples_split 40: 0.733519 (0.112683)
DT performance tuning min_samples_split 45: 0.733519 (0.112683)
DT performance tuning min_samples_split 50: 0.660539 (0.143192)
DT performance tuning min_samples_split 55: 0.598237 (0.132966)
DT performance tuning min_samples_split 60: 0.485625 (0.065475)
DT performance tuning min_samples_split 65: 0.485625 (0.065475)
DT performance tuning min_samples_split 70: 0.485625 (0.065475)
DT performance tuning min_samples_split 75: 0.485625 (0.065475)
DT performance tuning min_samples_split 8

It should be noted that the test-train split is randomized, and each random split is different.  Decision tree classifiers are known to be sensitive to the data, which means that we have to expect that slightly different randomizations of the training set will result in possibly drastic differences in performance.  

From the above tuning, it seems that setting the `min_samples_split=20` gives reliably best training set error.  Now we have to see how that works on the validation set that was reserved for evaluation only:

In [11]:
# Use the best value for min_samples_split from previous cell
choice = 20

# cross validation to evaluate the performance 
cv_results = model_selection.cross_val_score(DecisionTreeClassifier(min_samples_split=choice), 
                                             X_test, Y_test, cv=kfold, scoring=metric_name)

#displaying the mean and standard deviation of the prediction
msg = "%s %d: %f (%f)" % ('DT performance using tuned value for min_samples_split', choice, cv_results.mean(), cv_results.std())

print(msg)


DT performance using tuned value for min_samples_split 20: 0.891614 (0.115352)


The performance value obtained on the validation set is usually close to the value obtained on the training set while tuning, but the performance is not as stable as one might like. 

Just for fun, let's see what happens when we apply this tuned value to the whole data set:

In [12]:
# cross validation to evaluate the performance 
cv_results = model_selection.cross_val_score(DecisionTreeClassifier(min_samples_split=choice), 
                                             X, Y, cv=kfold, scoring=metric_name)

#displaying the mean and standard deviation of the prediction
msg = "%s %d: %f (%f)" % ('DT performance using tuned value for min_samples_split', choice, cv_results.mean(), cv_results.std())

print(msg)


DT performance using tuned value for min_samples_split 20: 0.844258 (0.076784)


This is better than predicted by the validation set, but still not as good as either Naive Bayes or KNN.  Decision trees are known to be sensitive to data.  Randomization of the train-test split results in fairly wide variance in performance.


### Grading
For full marks:
* You fitted the Decision Tree Classifier with a deliberately chosen options.
   * The best way to fit options is using a formal procedure like the above.
   * There's no need to fit all the possible options.  
   * An informal attempt to find good values for the options by giving a few values a try is less good.
* You calculated the $f_1$ score on the data
   * A validation set is the best way
   * Using cross-validation with the whole data set is less good.

## Discussion
**Methodology** The data was split into two equal halves, training and validation.  Tuning options and settings was done using the training set, and 10-fold cross validation.  Once the tuned settings and options were chosen, the method was applied to the validation set, and the performance evaluated using 10-fold cross validation.

Summarizing the results presented above, rounded to 3 significant figures (because more digits cloud the comparison without adding useful precision).

| Method | Options | Validation | Full |
| - |  |  |  |
| K Nearest Neighbours | $k=7$                   | 0.960 | 0.967 |
| Naive Bayes          | (none)                  | 0.956 | 0.978 |
| Decision Tree        | `min_samples_split` = 20| 0.852 | 0.854 |

We can see that
1. KNN had the best performance on the validation set, but only by a little.
1. Naive Bayes had the best performance on the full dataset, but only by a little.
1. Decision trees had the worst performance on the validation set and the full dataset.

Looking at the difference between the performance on the validation set compared to the full data set, it seems that performance improves a little with the full data set.  This makes sense in two ways.  First, more data means there is less chance to find accidental patterns that causeclassification errors.  Also, for KNN and DT, using the data with which we tuned the parameters means we're probably overfitting a little.  On the other hand, the difference between these results is small, so the over-fitting is a minor issue.  

Decision tree classification had trouble with this dataset because the separation between features was not easily represented in terms of vertical or horizontal splits in the feature space (see Question 2).  

In terms of effort to produce these results:
1. Naive Bayes was the simplest.
1. Decision Trees was the most trouble.

If best possible classification performance is the criterion, then it's a toss up between KNN and NB.  The difference is not appreciable unless there is a lot of money on the line with every error.  

Naive Bayes models are small and fast: it's quick to fit, and quick to use for classification.  KNN is trivial to fit, but troublesome to tune, and classification requires storage of the training dataset.  No summarization of the dataset at assll.  Decision trees are slow to fit, but fast to classify, but require a lot of tuning.

I choose Naive Bayes for this problem.

## Grading
The main issue here is that there are so many different ways this question could go, depending on what methods and options were used.  For example:
* There are 3 `f1` scoring options: `f1_micro`, `f1_macro`, and `f1_weighted`.
    * I had trouble with  `f1_macro`; its results were unpredictable and inscrutible.
* Using cross-validation or using simple test-train splits
* The size of the splits, and the number of folds.
    * I didn't notice a difference between 10-fold or 5-fold, but lots of variation with the test-train split at different proportions.
* Using `KFold()` for cross-validation vs `cv=10` in the cross-validation method.
    * using `cv=10` defaults **silently** to Strsatified Cross-validation, whereas `KFold()` does not.  

For these reasons, the reported $f_1$ scores could be quite diffrent from what I have reported.  
For full marks:
* Your Notebook is neat, and presents the problem and solution well
* You compared the three classifiers in terms of the $f_1$ score.
* Optional: You made other comparisons, e.g., ease of training, tuning, and computational performance issues.
